In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
import os
import pandas as pd

In [ ]:
PATH = "/content/drive/MyDrive/RSNA"
TRAINX_PATH = os.path.join(PATH,"train")
TRAINY_PATH = os.path.join(PATH,"train_labels.csv")

In [ ]:
trainy_file = pd.read_csv(TRAINY_PATH)
trainy_file.set_index('BraTS21ID',inplace = True)

In [ ]:
trainy_file

,MGMT_value
BraTS21ID,
0,1
2,1
3,0
5,1
6,1
...,...
1005,1
1007,1
1008,1


In [ ]:
import glob
import cv2
import numpy as np
trainx_data = []
i = 0
layers = 0
for folder in sorted(glob.glob(TRAINX_PATH+"/*")):
  # if (i<38):
    # i = i+1
    # continue
  if (i>=80):
    break
  print(folder)
  folder_data = []
  for file in sorted(glob.glob(folder+"/*.png")):
    try:
      img = cv2.imread(file)
    except:
      print("No image")
    folder_data.append(img)
  try:
    height, width, layers = folder_data[0].shape
  except:
    print("No list")
  size = (width, height)
  out = cv2.VideoWriter("data_new_"+str(i)+".mp4", cv2.VideoWriter_fourcc(*'DIVX'), 15, size)
  for j in range(len(folder_data)):
    out.write(folder_data[j])
  out.release()
  trainx_data.append(folder_data)
  i = i+1

/content/drive/MyDrive/RSNA/train/00000
/content/drive/MyDrive/RSNA/train/00002
/content/drive/MyDrive/RSNA/train/00003
/content/drive/MyDrive/RSNA/train/00005
/content/drive/MyDrive/RSNA/train/00006
/content/drive/MyDrive/RSNA/train/00008
/content/drive/MyDrive/RSNA/train/00009
/content/drive/MyDrive/RSNA/train/00011
/content/drive/MyDrive/RSNA/train/00012
/content/drive/MyDrive/RSNA/train/00014
/content/drive/MyDrive/RSNA/train/00017
/content/drive/MyDrive/RSNA/train/00018
/content/drive/MyDrive/RSNA/train/00019
/content/drive/MyDrive/RSNA/train/00020
/content/drive/MyDrive/RSNA/train/00021
/content/drive/MyDrive/RSNA/train/00022
/content/drive/MyDrive/RSNA/train/00024
/content/drive/MyDrive/RSNA/train/00025
/content/drive/MyDrive/RSNA/train/00026
/content/drive/MyDrive/RSNA/train/00028
/content/drive/MyDrive/RSNA/train/00030
/content/drive/MyDrive/RSNA/train/00031
/content/drive/MyDrive/RSNA/train/00032
/content/drive/MyDrive/RSNA/train/00033
/content/drive/MyDrive/RSNA/train/00035


In [ ]:
len(trainx_data)

80

In [ ]:
seq_len = 15
img_height = 64
img_width = 64
def frames_extraction(video_path):
    frames_list = []
     
    vidObj = cv2.VideoCapture(video_path)
    # Used as counter variable 
    count = 1
 
    while count <= seq_len: 
         
        success, image = vidObj.read() 
        if success:
            image = cv2.resize(image, (img_height, img_width))
            frames_list.append(image)
            count += 1
        else:
            print("Defected frame")
            # cv2.imwrite("frame%d.jpg" % count, image)
            break
 
            
    return frames_list

In [ ]:
X = []
y_csv = trainy_file["MGMT_value"]
Y = []
for i in range(0, 80):
  frames = frames_extraction("data_new_"+str(i)+".mp4")
  # print(len(frames))
  print(len(frames))
  if(len(frames)!=0):
    X.append(frames)
    Y.append(y_csv.iloc[i])
  # print(type(frames))
X = np.asarray(X)
Y = np.asarray(Y)

15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15


In [ ]:
X.shape

(80, 15, 64, 64, 3)

In [ ]:
Y.shape

(80,)

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.40, shuffle=True, random_state=0)

In [ ]:
from tensorflow import keras
from tensorflow.keras.layers import *
model = keras.Sequential()
model.add(ConvLSTM2D(filters = 64, kernel_size = (3, 3), return_sequences = False, data_format = "channels_last", input_shape = (seq_len, img_height, img_width, 3)))
model.add(Dropout(0.2))
model.add(Flatten())
model.add(Dense(256, activation="relu"))
model.add(Dropout(0.3))
model.add(Dense(6, activation = "softmax"))

In [ ]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv_lstm2d_1 (ConvLSTM2D)  (None, 62, 62, 64)        154624    
                                                                 
 dropout_2 (Dropout)         (None, 62, 62, 64)        0         
                                                                 
 flatten_1 (Flatten)         (None, 246016)            0         
                                                                 
 dense_2 (Dense)             (None, 256)               62980352  
                                                                 
 dropout_3 (Dropout)         (None, 256)               0         
                                                                 
 dense_3 (Dense)             (None, 6)                 1542      
                                                                 
Total params: 63,136,518
Trainable params: 63,136,518


In [ ]:
import tensorflow
opt = tensorflow.keras.optimizers.SGD(learning_rate=0.001)
model.compile(loss='sparse_categorical_crossentropy', optimizer=opt, metrics=["accuracy"])

In [ ]:

# !pip install tensorflow==2.4.1

In [ ]:
tensorflow.keras.__version__

'2.7.0'

In [ ]:
earlystop = tensorflow.keras.callbacks.EarlyStopping(patience=7)
callbacks = [earlystop]

In [ ]:
X_train.shape

(48, 15, 64, 64, 3)

In [ ]:
history = model.fit(x = X_train, y = y_train, epochs=5, batch_size = 8,  shuffle=True, callbacks=callbacks)

Epoch 1/5
6/6 [==============================] - 5s 582ms/step - loss: 0.2798 - accuracy: 0.9375
Epoch 2/5
6/6 [==============================] - 4s 585ms/step - loss: 0.1788 - accuracy: 0.9792
Epoch 3/5
6/6 [==============================] - 4s 587ms/step - loss: 0.2181 - accuracy: 0.9583
Epoch 4/5
6/6 [==============================] - 4s 589ms/step - loss: 0.1561 - accuracy: 1.0000
Epoch 5/5
6/6 [==============================] - 4s 587ms/step - loss: 0.1257 - accuracy: 1.0000


In [ ]:
from sklearn.metrics import classification_report
y_pred = model.predict(X_test)
y_pred = np.argmax(y_pred, axis=1)
print(y_pred)
print("---")
print(y_test)
print(classification_report(y_test, y_pred))

[1 1 0 1 1 0 0 0 0 1 1 1 0 0 1 1 1 1 0 1 0 1 0 0 0 1 1 1 0 1 0 1]
---
[1 0 1 1 1 1 0 1 1 1 1 0 0 0 0 1 1 0 1 1 0 1 1 0 0 1 1 1 0 1 0 1]
              precision    recall  f1-score   support

           0       0.57      0.67      0.62        12
           1       0.78      0.70      0.74        20

    accuracy                           0.69        32
   macro avg       0.67      0.68      0.68        32
weighted avg       0.70      0.69      0.69        32

